In [ ]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa
from imgaug import parameters as iap

# Paths to the original image directories
middle_phase_dir = 'Breast_cancer\Middle_Phase'
early_phase_dir = 'Breast_cancer\Early_Phase'

# Path to the augmented image directories
middle_phase_augmented_dir = 'Breast_cancer\Middle_phase_Augmented'
early_phase_augmented_dir = 'Breast_cancer\Early_phase_Augmentated'

# Create the directories for saving augmented images if they don't exist
os.makedirs(middle_phase_augmented_dir, exist_ok=True)
os.makedirs(early_phase_augmented_dir, exist_ok=True)

# Define the augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(1),  # Horizontal flip with 50% probability
    iaa.Rotate((-45, 45)),  # Random rotations between -45 to 45 degrees
    iaa.GaussianBlur(sigma=(0, 2.0)),  # Gaussian blur with sigma between 0 and 2.0
    iaa.AdditiveGaussianNoise(scale=(0, 0.2*255)),  # Additive Gaussian noise
    iaa.Dropout(p=(0, 0.2)),  # Random dropout with a probability between 0 and 0.2
    iaa.Resize({"height": (0.5, 1.5), "width": (0.5, 1.5)}),  # Random scaling of height and width
    iaa.Crop(percent=(0, 0.2)),  # Random cropping with up to 20% removal
    iaa.ElasticTransformation(alpha=(0, 10.0), sigma=1.0),  # Elastic transformation
    iaa.PiecewiseAffine(scale=(0.02, 0.1)),  # Piecewise affine transformation
    iaa.PerspectiveTransform(scale=(0.05, 0.15)),  # Perspective transformation
    iaa.LinearContrast((0.2, 3.0)),  # Contrast normalization
    iaa.Multiply((0.5, 1.5), per_channel=0.5),  # Random brightness adjustment
], random_order=True)  # Apply augmentations in random order

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
    return images

# Function to apply augmentations and save the augmented images
def augment_and_save_images(images, save_dir, num_augmented_images=6000):
    augmented_images = 0
    while augmented_images < num_augmented_images:
        # Apply augmentations
        augmented_batch = seq(images=images)
        for i, augmented_image in enumerate(augmented_batch):
            if augmented_images >= num_augmented_images:
                break
            # Save the augmented image to the specified directory
            save_path = os.path.join(save_dir, f"augmented_{augmented_images+1}.jpg")
            cv2.imwrite(save_path, augmented_image)
            augmented_images += 1

# Load original images from Middle_phase folder
middle_phase_images = load_images_from_folder(middle_phase_dir)

# Apply augmentations and save to the specified folder
augment_and_save_images(middle_phase_images, middle_phase_augmented_dir)

# Load original images from Early_phase folder
early_phase_images = load_images_from_folder(early_phase_dir)

# Apply augmentations and save to the specified folder
augment_and_save_images(early_phase_images, early_phase_augmented_dir)

print("Augmentation process completed successfully!")
